In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm


In [2]:

#this function converts a single row of data from a dataframe into a biunarised list of amino acids.
def binarise(row):
    
    max_protein_size = 336
    
    amino_acid_types = 20
    
    
    blank_sequence = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    amino_acid_cipher = {'A': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'C': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'D': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'E': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'F': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'G': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'H': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'I': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'K': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'L': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'M': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'N': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
                         'P': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
                         'Q': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                         'R': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 
                         'S': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
                         'T': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
                         'V': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 
                         'W': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 
                         'Y': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}
    
    

    def sequencer(string_sequence):
        
        binarised_sequence = []
        
        string_sequence = list(string_sequence)
        
        
        for amino_acid in string_sequence:
            
            binarised_sequence += amino_acid_cipher[amino_acid]
            
        while len(binarised_sequence) < (max_protein_size*amino_acid_types):
            binarised_sequence += blank_sequence 
        
        return binarised_sequence
            
    
    first_sequence = row.loc['sequence_one']
    first_sequence = sequencer(first_sequence)
    
    
    second_sequence = row.loc['sequence_two']
    second_sequence = sequencer(second_sequence)
    
    final_sequence = first_sequence + second_sequence
    
    
    output = 0
    
    if row.loc['dtm'] > 0:
        output = 1

    
    return final_sequence, [output]
            
        
   

In [3]:
# list juicer [  [ [protein sequence pairs  making up 1s and 0s],  [...],  [...],  [...],   [...]   ],  [observed results]   ] 
# basically the inputs are a bunch of 1s and os for [pairs of proiteins

In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda:0")

In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(336*20*2, 336*20)
        self.layer_2 = nn.Linear(336*20, 336*20)
        self.layer_3 = nn.Linear(336*20, 336*10)
        self.layer_4 = nn.Linear(336*10, 500)
        self.layer_5= nn.Linear(500, 1)
        
        
        #self.layer_3 = nn.Linear(336*10, 1) 
    
    def forward(self, X):
        X = F.relu(self.layer_1(X))
        X = F.relu(self.layer_2(X))
        X = F.relu(self.layer_3(X))
        X = F.relu(self.layer_4(X))
        X = self.layer_5(X)
        
        return  torch.sigmoid(X)  

# net = torch.load("the_juice.pt").to(device) 
# net.eval()

net = Net().to(device) 
            

In [16]:
proteins = pd.read_csv("filteredtrain3.csv")

In [17]:
torch.seed()

109899396836300

In [32]:

def train(net):
    
    print('Our seed is: ', torch.seed(), '\n')
    
    criterion = nn.BCELoss()
    #optimiser = optim.Adam(net.parameters(), lr = 0.001)
    optimiser = optim.SGD(net.parameters(), lr = 0.001)
    BATCH_SIZE = 64
    EPOCHS = 100
    
    loss_list = []
    
    try:

        for epoch in range(EPOCHS):
            for i in tqdm(range(0, proteins.shape[0]-1, BATCH_SIZE)):


                optimiser.zero_grad()


                real_outputs =  []
                predicted_outputs = []

                batch_X = []

                batch_y = []

                for j in range(i, i+BATCH_SIZE):
                    if j < proteins.shape[0]:

                        X, y = binarise(proteins.loc[j])


                        batch_X.append(X)
                        batch_y.append(y)


                batch_X = torch.tensor(batch_X, dtype = torch.float).to(device)



                batch_y = torch.tensor(batch_y, dtype = torch.float).to(device)



                predicted_outputs = net(batch_X)

                loss = criterion(predicted_outputs, batch_y)
                loss.backward()
                optimiser.step()


            print(f'Epoch: {epoch+1}, Loss = {loss.item():.7f}', '\n')

            if len(loss_list) == 2:
                if (abs(loss_list[0] - loss_list[1]) <= 0.01) and (epoch > 10):
                    break

                else:
                    loss_list.clear()

            loss_list.append(loss.item())
                
    except KeyboardInterrupt:
        pass
    
    torch.save(net, "/Users/moham/OneDrive/Documents/enzymes/programs/models/the_juice11.pt")
            

In [33]:
train(net)

Our seed is:  116956905806700 



  0%|          | 0/3590 [00:00<?, ?it/s]

Epoch: 1, Loss = 0.6932697 



  0%|          | 0/3590 [00:00<?, ?it/s]

Epoch: 2, Loss = 0.6929655 



...

Epoch: 11, Loss = 0.6890938 



  0%|          | 0/3590 [00:00<?, ?it/s]

Epoch: 12, Loss = 0.6878598 



  0%|          | 0/3590 [00:00<?, ?it/s]

Epoch: 13, Loss = 0.6861303 

